In [1]:
%%writefile MatrixTranspose.cu
#include <cuda_runtime.h>
#include <iostream>

// Define the size of the matrix
#define WIDTH 1024
#define HEIGHT 1024

// CUDA kernel for matrix transposition
__global__ void transposeMatrix(const float* input, float* output, int width, int height) {
    // Calculate the row and column index of the element
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    // Perform the transposition if within bounds
    if (x < width && y < height) {
        int inputIndex = y * width + x;
        int outputIndex = x * height + y;
        output[outputIndex] = input[inputIndex];
        // Add print statement for demonstration
        if (x < 10 && y < 10) { // Print only for a small subset of threads
            printf("Thread (%d, %d) in Block (%d, %d): Transposing element (%d, %d)\n",
                   threadIdx.x, threadIdx.y, blockIdx.x, blockIdx.y, y, x);
        }
    }
}

// Host function to check for CUDA errors
void checkCudaError(const char* message) {
    cudaError_t error = cudaGetLastError();
    if (error != cudaSuccess) {
        std::cerr << message << " - CUDA Error: " << cudaGetErrorString(error) << std::endl;
        exit(EXIT_FAILURE);
    }
}

int main() {
    int width = WIDTH;
    int height = HEIGHT;

    // Allocate host memory
    size_t size = width * height * sizeof(float);
    float* h_input = (float*)malloc(size);
    float* h_output = (float*)malloc(size);

    // Initialize the input matrix with some values
    for (int i = 0; i < width * height; i++) {
        h_input[i] = static_cast<float>(i);
    }

    // Allocate device memory
    float* d_input;
    float* d_output;
    cudaMalloc((void**)&d_input, size);
    cudaMalloc((void**)&d_output, size);

    // Copy data from host to device
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);
    checkCudaError("Failed to copy input data to device");

    // Define block and grid sizes
    dim3 blockSize(32, 32);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch the kernel
    transposeMatrix<<<gridSize, blockSize>>>(d_input, d_output, width, height);
    cudaDeviceSynchronize();
    checkCudaError("Kernel execution failed");

    // Copy the result back to the host
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);
    checkCudaError("Failed to copy output data to host");

    // Verify the result and print some elements
    bool success = true;
    std::cout << "Printing a few elements of the transposed matrix:" << std::endl;
    for (int i = 0; i < 10; i++) { // Print first 10 rows of the transposed matrix
        for (int j = 0; j < 10; j++) { // Print first 10 columns
            if (h_output[i * height + j] != h_input[j * width + i]) {
                success = false;
            }
            std::cout << h_output[i * height + j] << " ";
        }
        std::cout << std::endl;
    }


    std::cout << (success ? "Matrix transposition succeeded!" : "Matrix transposition failed!") << std::endl;

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_output);

    // Free host memory
    free(h_input);
    free(h_output);

    return 0;
}

Writing MatrixTranspose.cu


In [2]:
# Compile with the specified architecture
!nvcc MatrixTranspose.cu -o MatrixTranspose -gencode arch=compute_75,code=sm_75

# Run the executable
!./MatrixTranspose

Thread (0, 6) in Block (0, 0): Transposing element (6, 0)
Thread (1, 6) in Block (0, 0): Transposing element (6, 1)
Thread (2, 6) in Block (0, 0): Transposing element (6, 2)
Thread (3, 6) in Block (0, 0): Transposing element (6, 3)
Thread (4, 6) in Block (0, 0): Transposing element (6, 4)
Thread (5, 6) in Block (0, 0): Transposing element (6, 5)
Thread (6, 6) in Block (0, 0): Transposing element (6, 6)
Thread (7, 6) in Block (0, 0): Transposing element (6, 7)
Thread (8, 6) in Block (0, 0): Transposing element (6, 8)
Thread (9, 6) in Block (0, 0): Transposing element (6, 9)
Thread (0, 3) in Block (0, 0): Transposing element (3, 0)
Thread (1, 3) in Block (0, 0): Transposing element (3, 1)
Thread (2, 3) in Block (0, 0): Transposing element (3, 2)
Thread (3, 3) in Block (0, 0): Transposing element (3, 3)
Thread (4, 3) in Block (0, 0): Transposing element (3, 4)
Thread (5, 3) in Block (0, 0): Transposing element (3, 5)
Thread (6, 3) in Block (0, 0): Transposing element (3, 6)
Thread (7, 3) 